In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import random
from random import randint
import warnings
import cv2
import time
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

### Porbability tables and arrays class

In [198]:
class PorbabilityClass():
    
   

    
    def __init__(self, aisle, weekday):
        self.weekday = weekday
        self.aisle = aisle
    
    
    def read_data(self):
        df = pd.read_csv('friday.csv', delimiter=";")
        for csv in os.listdir():
            if csv[-3:] == "csv":                
                df_day = pd.read_csv(f'{csv}', delimiter=";")
                df = pd.concat([df, df_day], sort=True)
        return df        
           
    
    def add_datetime_columns(self):
        df =  self.read_data()
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        df['hour'] = df['timestamp'].dt.hour
        df['weekday'] = df['timestamp'].dt.weekday
        df['weekday'].replace({0:'Mon',1:'Tues', 2:'Wednes',3:'Thurs',4:'Fri'}, inplace=True)
        
        return df
    
        
        
    def porb_matrix_by_day(self):
        df = self.add_datetime_columns()
        if self.weekday == 'all_days':       
            df_1 = pd.DataFrame(df.sort_values(['customer_no','timestamp']).groupby(['weekday','customer_no','location'])['timestamp'].min())
            df_1 = df_1.sort_values(["customer_no",'timestamp'])
            df_1 = pd.DataFrame(df_1.index.get_level_values(2),df_1['timestamp']).reset_index()
            
        else:
            df_1 = df[df['weekday']== self.weekday]
            df_1 = pd.DataFrame(df_1.sort_values(['customer_no','timestamp']).groupby(['customer_no','location'])['timestamp'].min())
            df_1 = df_1.sort_values(["customer_no",'timestamp'])
            df_1 = pd.DataFrame(df_1.index.get_level_values(1),df_1['timestamp']).reset_index()   


        df_1['next_location'] = df_1['location'].shift(-1)
        df_1['next_location'][df_1['location']=='checkout']= 'checkout'
        df_1['initial'] = df_1['location'].shift(-1)

        first_loc = df_1['initial'][df_1['location']=='checkout']
        first_loc = first_loc.value_counts()/first_loc.value_counts().sum()

        prob_mat = pd.crosstab(df_1['location'], df_1['next_location'], normalize=0)

        return first_loc, prob_mat 
    
        
        
        
    def sector_time_prob(self):
        df = self.add_datetime_columns()
        if self.weekday == 'all_days': 
            df_1 = pd.DataFrame(df.sort_values(['customer_no','timestamp']).groupby(['weekday','customer_no','location'])['timestamp'].min())
            df_1 = df_1.sort_values(["customer_no",'timestamp'])
            df_1= pd.DataFrame(df_1.index.get_level_values(2), df_1['timestamp']).reset_index()
        else:
            df_1 = df[df['weekday'] == self.weekday]
            df_1 = pd.DataFrame(df_1.sort_values(['customer_no','timestamp']).groupby(['customer_no','location'])['timestamp'].min())
            df_1 = df_1.sort_values(["customer_no",'timestamp'])
            df_1= pd.DataFrame(df_1.index.get_level_values(1), df_1['timestamp']).reset_index()
       
        df_1['timestamp-2'] = df_1['timestamp'].shift(-1)        
        df_1= df_1[df_1['location'] == self.aisle]
        df_1['timespent'] = df_1['timestamp-2'] - df_1['timestamp']
        df_1 = df_1[(df_1['timespent']< pd.to_timedelta('40m')) & (df_1['timespent'] >= pd.to_timedelta('1m'))]

        time_prob = df_1['timespent'].value_counts()/df_1['timespent'].value_counts().sum()
        time_prob.index = time_prob.index/pd.Timedelta('1 min')

        return time_prob
 

In [199]:
aisles_locs= {'fruit':(180,750), 'spices':(150,570), 'drinks':(280,140), 'dairy':(130,335), 'checkout':(600,220)}

### Customer Class

In [236]:
class Customer(PorbabilityClass):
    """ This is a customer class """
    
    def __init__(self, aisle, weekday, image, target_position):
        PorbabilityClass.__init__(self, aisle, weekday)
        self.location = np.array((randint(580, 620), randint(700, 780)))   
        self.image = image      
        self.h = self.image.shape[0]
        self.w = self.image.shape[1]
        self.payment = 0
        self.shopping_hist = []
        self.target_position = np.array(target_position)
          
    def aisle_pattern(self):
        #first time your customer arrives, place her in the 'first_aisle - every aisle except checkout'
        initial_state, prob_mat = self.porb_matrix_by_day()
        aisle = random.choices(initial_state.index, initial_state)[0]
        yield aisle
        #for every other time until she checkouts, place her wherever the aisle_activity prob matrix places her
        while True:
            aisle = random.choices(prob_mat.index, prob_mat.loc[aisle])[0]
            yield aisle
            
    def move(self):
        '''
        The method move lets the customers move in the supermarket
        '''
        y, x = self.location
   
        self.target_position = np.array(aisles_locs[self.shopping_hist[0]])
        ty, tx = self.target_position          
        # go up
        if x != tx and y != 60:
            trajectory = 60 - y
            if trajectory > 0:
                self.location[0] += 1
            elif trajectory < 0:
                self.location[0] -= 1

        # go right or left
        if x != tx and y == 60:
            trajectory = tx - x
            if trajectory > 0:
                self.location[1] += 1
            if trajectory < 0:
                self.location[1] -= 1

        # go down
        if x == tx:
            trajectory = ty - y
            if trajectory > 0:
                self.location[0] += 1
            if trajectory < 0:
                self.location[0] -= 1
                
#     def move_2(self):
#         '''
#         The method move lets the customers move in the supermarket
#         '''

# #       for i in range(len(self.shopping_hist)):
#         y, x = self.location
   
#         self.target_position = np.array(self.aisles_locs[self.shopping_hist[1]])
#         ty, tx = self.target_position          
#         # go up
#         if x != tx and y != 60:
#             trajectory = 60 - y
#             if trajectory > 0:
#                 self.location[0] += 1
#             elif trajectory < 0:
#                 self.location[0] -= 1

#         # go right or left
#         if x != tx and y == 60:
#             trajectory = tx - x
#             if trajectory > 0:
#                 self.location[1] += 1
#             if trajectory < 0:
#                 self.location[1] -= 1

#         # go down
#         if x == tx:
#             trajectory = ty - y
#             if trajectory > 0:
#                 self.location[0] += 1
#             if trajectory < 0:
#                 self.location[0] -= 1         
       
 

    def money_spent(self):
        for section in self.shopping_hist:
            if section == 'dairy':
                self.aisle = 'dairy'
                time_prob = self.sector_time_prob()
                time_spent = random.choices(time_prob.index, time_prob)[0]
                pay = 5 * time_spent
                self.payment  = self.payment + pay
            if section == 'fruit':
                self.aisle = 'fruit'
                time_prob = self.sector_time_prob()
                time_spent = random.choices(time_prob.index, time_prob)[0]
                pay = 4 * time_spent
                self.payment  = self.payment + pay
            if section == 'spices':
                self.aisle = 'spices'
                time_prob = self.sector_time_prob()
                time_spent = random.choices(time_prob.index, time_prob)[0]
                pay = 3 * time_spent
                self.payment  = self.payment + pay
            if section == 'drinks':
                self.aisle = 'drinks'
                time_prob = self.sector_time_prob()
                time_spent = random.choices(time_prob.index, time_prob)[0]
                pay = 6 * time_spent
                self.payment  = self.payment + pay     
            else:
                self.payment  = self.payment

    def pattern(self):        
        pattern_func = self.aisle_pattern()
        while True:
            next_step = next(pattern_func)
            self.shopping_hist.append(next_step)
            if next_step =='checkout':
                break


In [242]:
class SupermarketSim(Customer):
    
    def __init__(self,customers,background=cv2.imread('market.png')):
#         super().__init__(self, target_position)
        self.background = background
        self.customers = customers
        self.frame = background
        
    def draw(self):
        self.frame = self.background.copy()
        for customer in customers:
            y, x = customer.location
            self.frame[y:y+customer.h, x:x+customer.w] = customer.image

    def run_one_iteration(self):
        for customer in customers:
            customer.move()
#             if self.location[0] ==aisles_locs[customers[1].shopping_hist[0]][0] and self.location[1] == aisles_locs[customers[1].shopping_hist[1]]:                
#                 customer.move_2()
            
            


In [243]:
customers = []
for _ in range(2):
    c = Customer(weekday='all_days', aisle='fruit', image= cv2.imread('customer_2.png'), target_position = (280, 140))
    c.pattern()
    c.money_spent()
    customers.append(c)


C:\Users\Khamirz\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [244]:
aisles_locs[customers[1].shopping_hist[0]]

(130, 335)

In [245]:
customers[1].move()

In [246]:
sim = SupermarketSim(customers=customers)

In [247]:
img = cv2.imread('market.png')

In [248]:
while True:
    frame = img.copy()

    sim.draw()
    sim.run_one_iteration()


    cv2.imshow('frame', sim.frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cv2.destroyAllWindows()
